<a href="https://colab.research.google.com/github/cammylexi/CS2341-Assignment-3/blob/main/Lab3Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Overview

The goal of this lab is to develop a machine learning model to classify ships based on 3 multi-class classification tasks using Ship Performance Dataset. The tasks include predicting the Ship Type, Route Type, and Maintenance Status based on numerical and catergorical features. With these tasks we can help optimize fleet operations, improvr cost efficiency, and ensure regulatory compliance in maritime industry.  

<br>

1. Predicting Ship Type

Task and Use Case
- The different ships are classified as Tanker, Container SHhp, Fish Carrier, and Bulk Carrier. This can help with fleet optimizations because if different ship types are suited for different cargo and routes, so classification and help with making good operational decisions. In addition to managing cost takes understanding of the type of ship and all that it needs llike the type of fuel engine effiencieny and operational cost.

Interested Parties
-  Shipping Companies and Maritime Logistics would be interested in this model because of its ability to optimize fleet deployment and to improve efficiency to increase their revenue.  

<br>
2. Predicting Route Type

Task and Use Case
- The ship's route is classified inro Short-haul, Long-haul, or Transoceanic which is based on operational characteristics such as speed, engine power and fuel usage. Different route have varying fuel costs, risks and operational constraints. The risk on the route would be accessed based on the on the weather and maintainence. Usually, longer routes leave ships exposed to harsher weather condition and higher maintenance needs. If shipping companies know the route type that is taken, the model can help them optimize voyage planning to planning reduce cost and improve efficiency.

Interested Parties
- Insurance companies would be interested in this model to assess the risk of the voyage and adjust the insurance policies accordingly.
- Shipping companies would also be benfit from this because they would be able to make informed decisions about fuel cost and operational risks.  

<br>
3. Predicting Maintainence Status


Task and Use Case
- A ship's maintenance condition is Fair, Critical, or Good and can be classifed using data on the engine performance, distance traveled, and fuel efficiency. Sometimes during voyage unexpected breakdowns happen and can be costly, which can lead to delays, lost in revenue, and even safety risks. A predictive maintainence model allows compaines to schedule repairs before failures occur and to avoid unnecessary expenses.

Interested Parties
-  Shipowners and Fleet managers would benefit from this so they can proactively schedule maintainance and reduce unexpected repair cost.
- Maritime Insurance Companies would use this to assess risk based maintainence conditions and adjust insurance coverage.

<br>

These classifications are important to provide insight for decision-making for shipping companies. The Ship Type and Route Type models are more suitable for offline analysis, because over time logistics teams can analyze patterns from past voyages and improve route planning for the nex shipping. On the other hand, Maintainence Status can be deployed for real-time predictions, because the classification would come immediately to the fleet manager to take action.

In order for these models to be useful they must give a high precision and recall, especially for the Maintainance Status prediction. In the case, a false negative occurs -- where a "Fair" condition is called instead of "Critical" -- could lead to serious safety issues and high expenses.






#Data Preparation and Preprocessing

## Defining Class Variables

###Numerical Features (Continuous Variables)
- Speed_Over_Ground_knots (float): Average speed of the ship over water (in knots).

- Engine_Power_kW (float): Engine power output (in kilowatts).
Distance_Traveled_nm (float): Total distance traveled by the ship (in nautical miles).

- Cargo_Weight_tons (float): Total weight carried by the ship (in tons).

- Operational_Cost_USD (float): Total operational cost per voyage (in USD).

- Revenue_per_Voyage_USD (float): Revenue generated per voyage (in USD).

- Efficiency_nm_per_kWh (float): Energy efficiency calculated in nautical miles per kilowatt-hour.

###Categorical Features (Discrete Variables)
- Ship_Type (categorical): Type of ship (e.g., Tanker, Container Ship, Fish Carrier, Bulk Carrier).

- Route_Type (categorical -> integer encoding {0, 1, 2, 3, 4}): Shipping route type (e.g., Short-haul, Long-haul, Transoceanic).

- Engine_Type (categorical): Type of engine (e.g., Diesel, Heavy Fuel Oil).

- Maintenance_Status (categorical -> ordinal encoding {3:Good, 2:Fair, 1:Critical, NaN:None}): Maintenance condition of the ship (e.g., Fair, Critical, Good).


##Data Relevance
Each attribute in the dataset contributes to classification of Ship Type, Route Type, and Maintenance Status.

- Operational Efficiency: By examining Speed_Over_Ground_knots, Efficiency_nm_per_kWh, and Engine_Power_kW,Distance_Traveled_nm.

- Economic Viability: Through Operational_Cost_USD and Revenue_per_Voyage_USD.

- Predictive Modeling: Using Ship_Type, Route_Type, and Engine_Type to classify ships and estimate key performance metrics.

- Risk Assessment: Based on Maintenance_Status and Weather_Condition, which helps determine potential breakdown and natural risks.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline

print('Pandas:', pd.__version__)
print('Numpy:',np.__version__)

#this is our data at this link "https://github.com/LLANES-MA/MLPythonLab1"
df = pd.read_csv('https://raw.githubusercontent.com/LLANES-MA/MLPythonLab1/refs/heads/main/Ship_Performance_Dataset.csv')

for col in ['Date','Draft_meters','Turnaround_Time_hours', 'Seasonal_Impact_Score', 'Weekly_Voyage_Count', 'Average_Load_Percentage']:
    if col in df:
        del df[col]

df.head()

Pandas: 2.2.2
Numpy: 1.26.4


,Ship_Type,Route_Type,Engine_Type,Maintenance_Status,Speed_Over_Ground_knots,Engine_Power_kW,Distance_Traveled_nm,Weather_Condition,Cargo_Weight_tons,Operational_Cost_USD,Revenue_per_Voyage_USD,Efficiency_nm_per_kWh
0,Container Ship,NaN,Heavy Fuel Oil (HFO),Critical,12.597558,2062.983982,1030.943616,Moderate,1959.017882,483832.354540,292183.273104,1.455179
1,Fish Carrier,Short-haul,Steam Turbine,Good,10.387580,1796.057415,1060.486382,Rough,162.394712,483388.000509,883765.787360,0.290361
2,Container Ship,Long-haul,Diesel,Fair,20.749747,1648.556685,658.874144,Moderate,178.040917,448543.404044,394018.746904,0.499595
3,Bulk Carrier,Transoceanic,Steam Turbine,Fair,21.055102,915.261795,1126.822519,Moderate,1737.385346,261349.605449,87551.375175,0.702906
4,Fish Carrier,Transoceanic,Diesel,Fair,13.742777,1089.721803,1445.281159,Moderate,260.595103,287718.375160,676121.459632,1.331343


##Removed Variables

We deleted variables Date, Draft_Meters, Turnaround_Time_hours,Seasonal_Impact_Score, Weekly_Voyage_Count, Average_Load_Percentage. [reasoning]

In [2]:
#first let us change the numeric values, where appropriate, to numeric using the .map() function
mStatusMap = {'Good':3, 'Fair':2, 'Critical':1, 'None':np.nan}
df['Maintenance_Status'] = df['Maintenance_Status'].map(mStatusMap)

wConditionMap = {'Rough':3, 'Moderate':2, 'Calm':1, 'None':np.nan}
df['Weather_Condition'] = df['Weather_Condition'].map(wConditionMap)

df.head()

,Ship_Type,Route_Type,Engine_Type,Maintenance_Status,Speed_Over_Ground_knots,Engine_Power_kW,Distance_Traveled_nm,Weather_Condition,Cargo_Weight_tons,Operational_Cost_USD,Revenue_per_Voyage_USD,Efficiency_nm_per_kWh
0,Container Ship,NaN,Heavy Fuel Oil (HFO),1.0,12.597558,2062.983982,1030.943616,2.0,1959.017882,483832.354540,292183.273104,1.455179
1,Fish Carrier,Short-haul,Steam Turbine,3.0,10.387580,1796.057415,1060.486382,3.0,162.394712,483388.000509,883765.787360,0.290361
2,Container Ship,Long-haul,Diesel,2.0,20.749747,1648.556685,658.874144,2.0,178.040917,448543.404044,394018.746904,0.499595
3,Bulk Carrier,Transoceanic,Steam Turbine,2.0,21.055102,915.261795,1126.822519,2.0,1737.385346,261349.605449,87551.375175,0.702906
4,Fish Carrier,Transoceanic,Diesel,2.0,13.742777,1089.721803,1445.281159,2.0,260.595103,287718.375160,676121.459632,1.331343


Data Quality (After handling missing data)

In [3]:
# Elimating: ship_type, engine_type, route_type
# Impute: maintenance status, weather conditions
df_grouped = df.groupby(by='Ship_Type')

import math
# Code originally written and revised from Dr. Larson
func = lambda grp: grp.fillna(int(grp.median())) # within groups, fill using median (define function to do this)
numeric_columns = ['Maintenance_Status', 'Weather_Condition'] # only transform numeric columns
df_imputed = df_grouped[numeric_columns].transform(func) # apply impute and transform the data back

# Extra step: fill any object columns that could not be transformed
col_deleted = list( set(df.columns) - set(df_imputed.columns)) # in case the median operation deleted columns
df_imputed[col_deleted] = df[col_deleted]

# drop any rows that still had missing values after grouped imputation
df_imputed.dropna(inplace=True)

# Ensure Weather_Condition and Maintenance Status remain categorical (discrete values)
df_imputed['Weather_Condition'] = df_imputed['Weather_Condition'].astype(int)
df_imputed['Maintenance_Status'] = df_imputed['Maintenance_Status'].astype(int)

# 5. Rearrange the columns
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2354 entries, 1 to 2735
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Maintenance_Status       2354 non-null   int64  
 1   Weather_Condition        2354 non-null   int64  
 2   Speed_Over_Ground_knots  2354 non-null   float64
 3   Distance_Traveled_nm     2354 non-null   float64
 4   Engine_Type              2354 non-null   object 
 5   Cargo_Weight_tons        2354 non-null   float64
 6   Revenue_per_Voyage_USD   2354 non-null   float64
 7   Ship_Type                2354 non-null   object 
 8   Operational_Cost_USD     2354 non-null   float64
 9   Efficiency_nm_per_kWh    2354 non-null   float64
 10  Route_Type               2354 non-null   object 
 11  Engine_Power_kW          2354 non-null   float64
dtypes: float64(7), int64(2), object(3)
memory usage: 239.1+ KB


##Final Dataset

[Description}bold text

##After Preprocessed Data

In [4]:
df.describe()

,Maintenance_Status,Speed_Over_Ground_knots,Engine_Power_kW,Distance_Traveled_nm,Weather_Condition,Cargo_Weight_tons,Operational_Cost_USD,Revenue_per_Voyage_USD,Efficiency_nm_per_kWh
count,2600.00000,2736.000000,2736.000000,2736.000000,2600.000000,2736.000000,2736.000000,2736.000000,2736.000000
mean,2.00500,17.602863,1757.610939,1036.406203,1.970385,1032.573264,255143.344488,521362.062011,0.798656
std,0.81656,4.311979,717.002777,568.632080,0.810360,558.697499,140890.482189,271211.625499,0.403590
min,1.00000,10.009756,501.025220,50.433150,1.000000,50.229624,10092.306318,50351.814447,0.100211
25%,1.00000,13.928452,1148.104914,548.511573,1.000000,553.983629,131293.378767,290346.388017,0.463592
50%,2.00000,17.713757,1757.494344,1037.816060,2.000000,1043.207497,257157.653226,520176.931762,0.789877
75%,3.00000,21.284785,2382.594292,1540.934161,3.000000,1527.722391,381796.930499,750072.791216,1.147426
max,3.00000,24.997043,2998.734329,1998.337057,3.000000,1999.126697,499734.867868,999916.696140,1.499259


#Modeling

#Deployment

#Exceptional Work